Launch a Dask backend for parallel computing:

In [2]:
from dask.distributed import Client

Client()

Client Scheduler: tcp://127.0.0.1:34609 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 16.66 GB


Change CWD to avoid `..` in file paths:

In [3]:
import os

os.chdir("/home/data-scientist/elo-merchant-category-recommendation")

Import libraries required across the whole notebook:

In [4]:
import dask.dataframe as dd

Load the data:

In [ ]:
dtype = dict()

In [9]:
import datetime
import numpy as np
import pandas as pd

In [14]:
%%time

category_2_dtype = pd.api.types.CategoricalDtype([-1, 1, 2, 3, 4, 5])
category_3_dtype = pd.api.types.CategoricalDtype([-1, 1, 2, 3])

dtype = dict(card_id="category",
             city_id="category",
             state_id="category",
             merchant_id="category",
             subsector_id="category",
             category_2=category_2_dtype,
             category_3=category_3_dtype)

kwargs = dict(parse_dates=["purchase_date"], dtype=dtype)

transactions = dict(old=dd.read_csv("data/v0-preprocessed/old/chunk.*.csv", **kwargs),
                    new=dd.read_csv("data/v0-preprocessed/new/chunk.*.csv", **kwargs))

CPU times: user 89 ms, sys: 1.31 ms, total: 90.3 ms
Wall time: 85.5 ms


In [15]:
%%time

for key, df in transactions.items():
    df["month_diff"] = df.month_lag + (datetime.datetime.today() - df.purchase_date).dt.days // 30
    df["purchase_month"] = df.purchase_date.dt.month # TODO: add_datepart (?)
    
    # try to leave category_2 column instead of / along with dummies
    transactions[key] = dd.get_dummies(df, columns=["category_2", "category_3"]) # TODO: dummy_na=True (?)

CPU times: user 19.1 ms, sys: 10.3 ms, total: 29.4 ms
Wall time: 26.4 ms


In [7]:
%%time

with pd.option_context("display.max_rows", 100):
    display(transactions["old"].isnull().any(axis=0).compute())

authorized_flag         False
card_id                 False
city_id                 False
category_1              False
installments            False
merchant_category_id    False
merchant_id              True
month_lag               False
purchase_amount         False
purchase_date           False
state_id                False
subsector_id            False
month_diff              False
purchase_month          False
category_2_-1           False
category_2_1            False
category_2_2            False
category_2_3            False
category_2_4            False
category_2_5            False
category_3_-1           False
category_3_1            False
category_3_2            False
category_3_3            False
dtype: bool

CPU times: user 4.98 s, sys: 370 ms, total: 5.35 s
Wall time: 37.4 s


In [18]:
%%time

agg = dict(# Percent of authorized transactions
           authorized_flag=["sum", "mean"],
           
           # Percent of transactions with positive category_1
           category_1=["sum", "mean"],
           
           # Percent of transactions by category_2
           category_2_1=["mean"],
           category_2_2=["mean"],
           category_2_3=["mean"],
           category_2_4=["mean"],
           category_2_5=["mean"],
           
           # Percent of transactions by category_3
           category_3_1=["mean"],
           category_3_2=["mean"],
           category_3_3=["mean"],
           
#            city_id     =["nunique"],
#            state_id    =["nunique"],
#            subsector_id=["nunique"],
           
#            merchant_id         =["nunique"],
#            merchant_category_id=["nunique"],
           
           purchase_amount=["mean", "max", "min", "std", "sum"],
           purchase_month =["mean", "max", "min", "std"],
#            purchase_date  =[np.ptp, "max", "min"],
    
           installments=["mean", "max", "min", "std", "sum"],

           month_lag =["mean", "max", "min", "std"],
           month_diff=["mean"])

for df in transactions.values():
    tmp = df.groupby("card_id").agg(agg).compute()
    
    display(tmp)
    
    break

authorized_flag           category_1           category_2_1  \
                            sum      mean        sum      mean         mean   
card_id                                                                       
C_ID_0046db9f8a             142  0.946667          3  0.020000     0.846667   
C_ID_011b0d9794              27  1.000000         27  1.000000     0.000000   
C_ID_01904d743d              50  0.704225          0  0.000000     1.000000   
C_ID_01b098ff01             128  0.703297         26  0.142857     0.851648   
C_ID_0382b662f4             162  0.952941          7  0.041176     0.947059   
C_ID_03d3f6c6f4              65  0.984848         47  0.712121     0.015152   
C_ID_058b254a5b             461  0.946612         57  0.117043     0.809035   
C_ID_05fce05831             105  0.867769          0  0.000000     0.000000   
C_ID_062c6dda0f              48  0.923077         34  0.653846     0.096154   
C_ID_06b77830e2             292  0.979866         36  0.120805     0.855705   
C_ID_094a0016ee             243  0.945525          0  0.000000     0.023346   
C_ID_0998bf9e51             465  0.993590         58  0.123932     0.854701   
C_ID_0ac3bdba13             105  0.889831          6  0.050847     0.940678   
C_ID_0d6b8c2d0f              35  0.972222          0  0.000000     1.000000   
C_ID_0d966d5bb3              97  0.915094          0  0.000000     0.990566   
C_ID_0e171c1b48             261  0.935484          0  0.000000     0.078853   
C_ID_0e4bbeaef2              25  0.735294          7  0.205882     0.000000   
C_ID_104b205261             294  0.976744         15  0.049834     0.950166   
C_ID_1170a4101e             145  0.917722         29  0.183544     0.810127   
C_ID_1275f28115             113  0.957627          4  0.033898     0.059322   
C_ID_12898d9f0c              35  0.897436         23  0.589744     0.410256   
C_ID_12b7811978              36  0.878049          0  0.000000     0.048780   
C_ID_149071be4f             201  0.980488         47  0.229268     0.019512   
C_ID_149d46d5d6             163  0.885870          0  0.000000     0.130435   
C_ID_14f06676e4              75  0.937500         14  0.175000     0.800000   
C_ID_190bd090c9              29  0.828571          2  0.057143     0.000000   
C_ID_1950177218             253  0.958333         86  0.325758     0.227273   
C_ID_1964c39b55             152  0.926829         32  0.195122     0.146341   
C_ID_198914cf94             623  0.912152          0  0.000000     0.055637   
C_ID_19d5c8812e             460  0.940695         29  0.059305     0.020450   
...                         ...       ...        ...       ...          ...   
C_ID_feb4db8b04              10  0.769231          4  0.307692     0.692308   
C_ID_feb7611669               5  1.000000          0  0.000000     1.000000   
C_ID_febc3b61b3              14  0.777778          0  0.000000     0.055556   
C_ID_febcbfc941              10  1.000000          0  0.000000     0.100000   
C_ID_fec9708598               9  1.000000          0  0.000000     0.777778   
C_ID_fedc1c215a               5  1.000000          0  0.000000     0.000000   
C_ID_feef470b4a               6  0.857143          0  0.000000     1.000000   
C_ID_ff12a8f4aa               9  1.000000          0  0.000000     1.000000   
C_ID_ff1648c840               6  0.750000          0  0.000000     0.000000   
C_ID_ff1ae05c84              16  1.000000          0  0.000000     0.062500   
C_ID_ff20d285f7               8  1.000000          2  0.250000     0.750000   
C_ID_ff28019c39              10  1.000000          0  0.000000     0.000000   
C_ID_ff2b2e5350               9  0.818182          0  0.000000     0.181818   
C_ID_ff2e2ff794               5  1.000000          5  1.000000     0.000000   
C_ID_ff30b86185               5  1.000000          5  1.000000     0.000000   
C_ID_ff3cd7bbea               9  0.692308          7  0.538462     0.461538   
C_ID_ff5f1abc3d               8  0.571429          1  0.071429    

CPU times: user 11.8 s, sys: 880 ms, total: 12.7 s
Wall time: 1min 47s


In [ ]:
data_dtype = dict(feature_1="category",
                  feature_2="category",
                  feature_3="category")

train_df = pd.read_csv("data/input/train.csv", dtype=data_dtype, parse_dates=["first_active_month"])
test_df  = pd.read_csv("data/input/test.csv",  dtype=data_dtype, parse_dates=["first_active_month"])

train_test_first_active_month = pd.concat([train_df.first_active_month, test_df.first_active_month])
test_df.first_active_month.fillna(train_test_first_active_month.value_counts().index[0], inplace=True)

train_df = train_df.merge(base_historical_transactions_df,            how="left", on="card_id") \
                   .merge(purchase_amount_historical_transactions_df, how="left", on="card_id")

test_df  = test_df.merge(base_historical_transactions_df,            how="left", on="card_id") \
                  .merge(purchase_amount_historical_transactions_df, how="left", on="card_id")

assert train_df.isnull().any().any() == False
assert  test_df.isnull().any().any() == False

display(train_df, test_df)

In [ ]:
train_df[["feature_1"]].astype(int).agg({ "feature_1_1": "mean" })

In [ ]:
for df in [train_df, test_df]:
    days_diff = train_df.first_active_month.max() - df.first_active_month
    df["years_since_first_active"]  = (days_diff / np.timedelta64(1, "Y")).astype(int)
    df["months_since_first_active"] = (days_diff / np.timedelta64(1, "M")).astype(int)
    
display(train_df, test_df)

In [ ]:
class AddYearAndMonthTransformer:
    def call(self, df):
        for attr in ["year", "month"]:
            df["first_active_month" + "_" + attr] = getattr(df.first_active_month.dt, attr)

        return df.drop("first_active_month", axis=1)
    
add_year_and_month_transformer = AddYearAndMonthTransformer()

train_df = add_year_and_month_transformer.call(train_df)
test_df  = add_year_and_month_transformer.call(test_df)

display(train_df, test_df)

In [ ]:
X_train, y_train = train_df.drop(["card_id", "target"], axis=1), train_df.target

X_test = test_df.drop("card_id", axis=1)

In [ ]:
%%time

model = RandomForestRegressor(n_estimators=100, max_features="log2", max_depth=10, n_jobs=-1, random_state=SEED)
scores = np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv=5))
print("CV: %.4f +/- %.4f" % (scores.mean(), scores.std() ** 2))

In [ ]:
%%time

model.fit(X_train, y_train)

In [ ]:
plt.scatter(y_train, model.predict(X_train), s=0.25);

In [ ]:
%%time

y_test = model.predict(X_test)

In [ ]:
submission_df = pd.read_csv("data/input/sample_submission.csv")
submission_df.target = y_test
submission_df.head()

In [ ]:
submission_df.isnull().any().any()

In [ ]:
submission_df.to_csv("submission.csv", index=False)

In [ ]:
!head submission.csv

In [ ]:
idx = np.argsort(model.feature_importances_)

importances = model.feature_importances_[idx]
columns = X_train.columns.values[idx]

plt.figure(figsize=(10, 10))
plt.barh(columns, importances);